<br><br>
<center><font size='5'><b>Deep SVDD (Support Vector Data Description)</b></font><br><br><font size='5'><b></b></font>

# 1. 이론

- anomaly detection을 하기 위해서는 latent space로 encoding을 잘해야 한다고 주장,
- 기존 AD의 경우 objective function이 이상치탐지를 위해 정의되지 않아서 좋은 성능을 내지 못한다고 주장.
- 저자가 만든 objective function을 사용하여 latent features 들을 hypersphere 안으로 encoding 하면 더 좋은 결과를 낸다.
  - hypersphere의 최종 반지름을 이용하면 추론 과정에서 decision boundary를 주관적으로 정해줘야 했던 기존 novelty detection 문제를 해결
- outlier 비율에 대한 사전 지식을 loss function에 넣어 soft-boundary Deep SVDD objective를 정의

## 1.1One-class classification?

- binary classification에서 class 한개의 데이터만 있고, 나머지 데이터는 클래스가 없는 경우.
- positive class 데이터셋에서만 학습을 하고 예측 함.

# 2. Paper review

- 논문명 : Deep one-class Classification (ICML 2018)

## 2.1. Introduction

- Classical AD 방법 : One-Class SVM, Kernel Density Estimation
  - 고차원에서 성능이 안좋고,성능 높이기 위해서 feature 뽑는 것이 중요했다.
- Deep learning으로 와서, 많은 AD 발전이 있었으나, __objective function 기반의 AD를 최적화하는 training 방법은 없었음.__
- Inspired by Kernel-based one-class classification / minimum volume estimation

![image](https://user-images.githubusercontent.com/48466625/61454209-36725b80-a99b-11e9-9f99-fb15943626ca.png)

- hypersphere의 볼륨을 최소화하면서 네트워크를 훈련시킨다.
  - common factor를 추출하는데 도움을 줄 것

## 2.2. Related work 

- kernel-based one-class classficiation (이것의 예제가 One-Class SVM)
- some Deep approaches 

### 2.2.1. Kernel-based One-class Classification

- 이 방법의 가장 대표적인 예제가 One-class SVM
- SVDD은 OC-SVM과 관련된 technique으로, hypersphere가 hyperplane 대신 데이터를 분리하는데에 사용된다.
  - feature 공간 상의 데이터를 포함하는 가장 작은 hypersphere를 찾는 것이 목적이다.
      - SVM에서 hyperplane이 가장 margin이 큰 경계선을 찾는 것과 비슷하게.
- 위 방법들은 기본적으로 dual prolem을 primal problem으로 구성하면서 풀 수 있는데,
  - kernel matrix를 구성하고 조작하는데서 오는 안좋은 computational scaling 문제가 있다.

### 2.2.2. Deep Approaches to Anomaly Detection

- deep learning 한 후에 classical AD 방법으로 하는 "mixed"
- fully learning directly 방법의 "fully deep"
  - reconstruction error에만 의존했던 다른 방법들(이 논문이 다른점)

[Deep Autoencoders]

- Model for inducing the network to extract salient features from dataset
- Reconstruction 에러를 최소화하게 train 되기 때문에,
  - 정상들은 common factor를 뽑고, 복원이 잘될 거지만, anomalous 들은 common factor를 포함하고 있지 않아 복원이 잘 안될것 
  - 그래서 mixed 방법으로 할 수도 있고, reconst error를 어노말리 스코어라 생각하고 fully deep으로도 사용할 수 있다.
- AE for AD
  - Denoising AE / Sparse AE / Variational AE / Deep Convolutional AE(가장 많이)
- AE를 AD에 사용하는데 가장 큰 문제점은 적당한 축소 정도(차원축소)를 선택하는 것이다.
  - which called "compactness", 비지도 학습에서 적당한 hyperparameter를 찾는 것은 어렵다.
  - Deep SVDD는 부피를 최소화하는 것으로 "compactness"를 고려하게 되었다.

## 3. Deep SVDD

### 3.1. Deep SVDD Objective

- NN that is jointly trained to map the data into hypersphere of minimum volume
- 인풋스페이스 -> 뉴럴넷(weight) -> 아웃풋 스페이스 에서, 
  - W를 학습 / output space에서 data를 enclosing하는 하이퍼스피어가 최소화되게끔
  
![image](https://user-images.githubusercontent.com/48466625/61456106-358ff880-a9a0-11e9-8208-c38610956fca.png)

- Deep SVDD 목적함수
- 볼륨최소화 / penalty term(allowing some pts to be mapped outside) / weight decay regularizer term
- 목적함수 -> 데이터포인트들이 하이퍼스피어의 center로 뭉치면서 mapping되는 W를 찾아준다.

![image](https://user-images.githubusercontent.com/48466625/61456828-ffec0f00-a9a1-11e9-8bfa-ae9262b96378.png)

- One-Class Deep SVDD 목적함수
- output space 상 데이터포인트와 중심 c간의 거리를 더해서 평균취한 값으로 해도 / 똑같이 최소 부피의 hypersphere를 찾는 것일 것
- 이 때 중심까지의 거리를 anomaly score라고 할 수 있을 것이며, 
  - 거리에서 최적 반지름 뺀것이 양수면 outlier일 것

### 3.2. Optimization of Deep SVDD

we train the network parameters W for some k ∈ N epochs
while the radius R is fixed. Then, after every k-th epoch,
we solve for radius R given the data representations from
the network using the network parameters W of the latest
update. R can be easily solved for via line search.


### 3.3. Properties of Deep SVDD.

trivial solution을 낼 수 있는 이 네트워크의 특성들을 이론적으로 증명함. 자세한 것은 논문 참고

## 4. Experiments (eval with MNIST and CIFAR10)

https://github.com/lukasruff/Deep-SVDD-PyTorch

- Shallow baselines : Kernel OC-SVM / SVDD, Kernel Density Estimation, Isolation Forest
- Deep baselines : DCAE, AnoGAN
- SVDD 
  - bias term 제거; hypersphere collapse 막기 위해서
  - line search every k=5 epochs
  - choose v from 0.001 ~ 0.1
  - center는 첫번째 forward 했을때 데이터의 평균으로 잡았다
  - AdamOptimizer
  - Batch Normalization
  - Initialization : DCAE는 uniform glorot weights를 한다음, DCAE 웨이트를 가져와서 초기화함(pre-training)
  - Leaky ReLU
  - 10개의 클래스니까, 총 10개의 one-class classification을 진행
  - global contrast normalization using L_1 norm하고, [0,1]로 min-max scaling 했음
  - LeNet-type CNN ; (conv - leaky ReLU - 2x2 maxpool) 2개의 모듈
  
![image](https://user-images.githubusercontent.com/48466625/61458788-46dc0380-a9a6-11e9-94ad-22bf1e8f328e.png)